In [1]:
import os
import itertools
import datetime as dt
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Noto Sans CJK JP'
import seaborn as sns
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from utils.helper import fix_seed, seed_worker
fix_seed()
from dataset import STDataset
from trainer import Trainer
from logger import Logger
import config

In [2]:
import loader

In [3]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=3)

## dfを前処理 → pickle形式のデータセットを作成

In [2]:
# !python3.9 df_preprocess.py ./datasets_1h/all.pkl -s 2021/4/2 -e 2022/6/1
# !python3.9 df_preprocess.py ./datasets_1h/mini.pkl -s 2021/4/2 -e 2021/6/1

In [3]:
# train
# !python3.9 df_preprocess.py ./datasets_1h/all.pkl -s 2021/4/2 -e 2022/3/1
# test
# !python3.9 df_preprocess.py ./datasets_1h/mini.pkl -s 2022/3/1 -e 2022/6/1

In [4]:
# mini
df_test = pd.read_pickle('./datasets_1h/kannetsu_210402-210531.pkl')
# whole
df_all = pd.read_pickle('./datasets_1h/kannetsu_210402-220531.pkl')

## Spatial Temporal Matrixに整形 & 標準化
- 区間数 x 時系列数 の行列
- 実際は 区間数 x 時系列数 x 特徴量数 のテンソル

- 標準化を行う
- (検索数, 台数は上り・下り別でもやってみる)

In [3]:
# !python3.9 transformer.py -s ./train.pkl ./features_train_norm.pkl ./labels_train.pkl --scaler_path ./datasets/training_scaler.pkl
# !python3.9 transformer.py -s ./test.pkl ./features_test_norm.pkl ./labels_test.pkl --scaler_path ./datasets/training_scaler.pkl

In [4]:
# train
# !python3.9 transformer.py -s ./datasets_1h/df_train.pkl ./datasets_1h/features_train_norm.pkl ./datasets_1h/labels_train.pkl --scaler_path ./datasets/training_scaler.pkl
# test
# !python3.9 transformer.py -s ./datasets_1h/df_test.pkl ./datasets_1h/features_test_norm.pkl ./datasets_1h/labels_test.pkl --scaler_path ./datasets/training_scaler.pkl

## 学習

In [12]:
!python3.9 train.py -h

usage: train.py [-h] [-e EPOCH] [-d {cpu,cuda,cuda:0,cuda:1,cuda:2,cuda:3}]
                [--model_name MODEL_NAME] [--log_path LOG_PATH]
                [--log_steps LOG_STEPS] [--background] [-r RANDOM_STATE]
                [--lstm_dropout LSTM_DROPOUT]
                {T-ED,ST-ED,TE-ED,STE-ED,STE-ED-S}

positional arguments:
  {T-ED,ST-ED,TE-ED,STE-ED,STE-ED-S}
                        training model

optional arguments:
  -h, --help            show this help message and exit
  -e EPOCH, --epoch EPOCH
                        number of epochs
  -d {cpu,cuda,cuda:0,cuda:1,cuda:2,cuda:3}, --device {cpu,cuda,cuda:0,cuda:1,cuda:2,cuda:3}
                        device on which training and inference are run
  --model_name MODEL_NAME
                        file name of saved model's parameter
  --log_path LOG_PATH   file name of model training log
  --log_steps LOG_STEPS
                        step size between epochs to output training logs
  --background          whether to output l

In [14]:
!python3.9 train.py STE-ED-S -d cuda:3 -e 10 -r 1000

============================== Training STE-ED-S for 10 epochs on cuda:3 (seed: 1000) ==============================
2022-10-10 08:12:04.715921 | Epoch: 1 | Train Loss: 886.051, Train Time: 6.42 [sec] | Valid Loss: 863.103, Valid Time: 0.91 [sec]
2022-10-10 08:12:09.911632 | Epoch: 2 | Train Loss: 725.203, Train Time: 4.29 [sec] | Valid Loss: 712.954, Valid Time: 0.90 [sec]
2022-10-10 08:12:15.159704 | Epoch: 3 | Train Loss: 637.516, Train Time: 4.31 [sec] | Valid Loss: 609.430, Valid Time: 0.93 [sec]
2022-10-10 08:12:20.412794 | Epoch: 4 | Train Loss: 466.276, Train Time: 4.32 [sec] | Valid Loss: 416.088, Valid Time: 0.93 [sec]
2022-10-10 08:12:25.677260 | Epoch: 5 | Train Loss: 329.638, Train Time: 4.36 [sec] | Valid Loss: 301.526, Valid Time: 0.90 [sec]
2022-10-10 08:12:30.931915 | Epoch: 6 | Train Loss: 244.340, Train Time: 4.35 [sec] | Valid Loss: 241.454, Valid Time: 0.91 [sec]
2022-10-10 08:12:36.226437 | Epoch: 7 | Train Loss: 184.896, Train Time: 4.39 [sec] | Valid Loss: 194.6

In [51]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_both, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_both, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/ST-ED_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:15:10.685435 | Epoch: 1 | Train Loss: 884.998, Train Time: 4.37 [sec] | Valid Loss: 862.037, Valid Time: 0.91 [sec]
2022-09-29 10:15:16.003705 | Epoch: 2 | Train Loss: 728.254, Train Time: 4.38 [sec] | Valid Loss: 723.285, Valid Time: 0.91 [sec]
2022-09-29 10:15:21.352083 | Epoch: 3 | Train Loss: 660.152, Train Time: 4.40 [sec] | Valid Loss: 659.807, Valid Time: 0.91 [sec]
2022-09-29 10:15:26.851966 | Epoch: 4 | Train Loss: 504.893, Train Time: 4.55 [sec] | Valid Loss: 442.367, Valid Time: 0.91 [sec]
2022-09-29 10:15:32.259302 | Epoch: 5 | Train Loss: 322.482, Train Time: 4.46 [sec] | Valid Loss: 299.131, Valid Time: 0.91 [sec]
2022-09-29 10:15:37.686665 | Epoch: 6 | Train Loss: 224.775, Train Time: 4.47 [sec] | Valid Loss: 222.983, Valid Time: 0.91 [sec]
2022-09-29 10:15:43.123238 | Epoch: 7 | Train Loss: 178.454, Train Time: 4.48 [sec] | Valid Loss: 186.756, Valid Time: 0.91 [sec]
2022-09-29 10:15:48.571414 | Epoch: 8 | Train Loss: 155.426, Train Time: 4.49 [sec] | Vali

In [52]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_context, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_context, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/ST-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:24:19.423752 | Epoch: 1 | Train Loss: 884.047, Train Time: 4.45 [sec] | Valid Loss: 858.919, Valid Time: 0.91 [sec]
2022-09-29 10:24:24.956937 | Epoch: 2 | Train Loss: 722.274, Train Time: 4.46 [sec] | Valid Loss: 712.109, Valid Time: 1.04 [sec]
2022-09-29 10:24:30.386196 | Epoch: 3 | Train Loss: 626.582, Train Time: 4.47 [sec] | Valid Loss: 574.777, Valid Time: 0.91 [sec]
2022-09-29 10:24:35.836868 | Epoch: 4 | Train Loss: 439.280, Train Time: 4.50 [sec] | Valid Loss: 386.793, Valid Time: 0.91 [sec]
2022-09-29 10:24:41.302459 | Epoch: 5 | Train Loss: 306.412, Train Time: 4.51 [sec] | Valid Loss: 272.567, Valid Time: 0.92 [sec]
2022-09-29 10:24:46.890899 | Epoch: 6 | Train Loss: 224.691, Train Time: 4.63 [sec] | Valid Loss: 230.020, Valid Time: 0.91 [sec]
2022-09-29 10:24:52.359531 | Epoch: 7 | Train Loss: 177.574, Train Time: 4.51 [sec] | Valid Loss: 187.574, Valid Time: 0.92 [sec]
2022-09-29 10:24:57.819853 | Epoch: 8 | Train Loss: 152.402, Train Time: 4.50 [sec] | Vali

In [52]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_context, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_context, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/ST-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:24:19.423752 | Epoch: 1 | Train Loss: 884.047, Train Time: 4.45 [sec] | Valid Loss: 858.919, Valid Time: 0.91 [sec]
2022-09-29 10:24:24.956937 | Epoch: 2 | Train Loss: 722.274, Train Time: 4.46 [sec] | Valid Loss: 712.109, Valid Time: 1.04 [sec]
2022-09-29 10:24:30.386196 | Epoch: 3 | Train Loss: 626.582, Train Time: 4.47 [sec] | Valid Loss: 574.777, Valid Time: 0.91 [sec]
2022-09-29 10:24:35.836868 | Epoch: 4 | Train Loss: 439.280, Train Time: 4.50 [sec] | Valid Loss: 386.793, Valid Time: 0.91 [sec]
2022-09-29 10:24:41.302459 | Epoch: 5 | Train Loss: 306.412, Train Time: 4.51 [sec] | Valid Loss: 272.567, Valid Time: 0.92 [sec]
2022-09-29 10:24:46.890899 | Epoch: 6 | Train Loss: 224.691, Train Time: 4.63 [sec] | Valid Loss: 230.020, Valid Time: 0.91 [sec]
2022-09-29 10:24:52.359531 | Epoch: 7 | Train Loss: 177.574, Train Time: 4.51 [sec] | Valid Loss: 187.574, Valid Time: 0.92 [sec]
2022-09-29 10:24:57.819853 | Epoch: 8 | Train Loss: 152.402, Train Time: 4.50 [sec] | Vali

In [50]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr_wo_search, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va_wo_search, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 10:05:55.620660 | Epoch: 1 | Train Loss: 887.670, Train Time: 4.27 [sec] | Valid Loss: 867.993, Valid Time: 0.90 [sec]
2022-09-29 10:06:00.940534 | Epoch: 2 | Train Loss: 732.711, Train Time: 4.39 [sec] | Valid Loss: 725.063, Valid Time: 0.90 [sec]
2022-09-29 10:06:06.189051 | Epoch: 3 | Train Loss: 624.695, Train Time: 4.31 [sec] | Valid Loss: 576.753, Valid Time: 0.90 [sec]
2022-09-29 10:06:11.461485 | Epoch: 4 | Train Loss: 443.205, Train Time: 4.33 [sec] | Valid Loss: 391.670, Valid Time: 0.90 [sec]
2022-09-29 10:06:16.872798 | Epoch: 5 | Train Loss: 302.375, Train Time: 4.47 [sec] | Valid Loss: 297.201, Valid Time: 0.90 [sec]
2022-09-29 10:06:22.165748 | Epoch: 6 | Train Loss: 232.168, Train Time: 4.35 [sec] | Valid Loss: 229.357, Valid Time: 0.90 [sec]
2022-09-29 10:06:27.490039 | Epoch: 7 | Train Loss: 190.038, Train Time: 4.38 [sec] | Valid Loss: 200.445, Valid Time: 0.91 [sec]
2022-09-29 10:06:32.841902 | Epoch: 8 | Train Loss: 169.546, Train Time: 4.40 [sec] | Vali

In [49]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/STE-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 09:55:16.926786 | Epoch: 1 | Train Loss: 886.051, Train Time: 6.26 [sec] | Valid Loss: 863.103, Valid Time: 0.90 [sec]
2022-09-29 09:55:22.106408 | Epoch: 2 | Train Loss: 725.203, Train Time: 4.25 [sec] | Valid Loss: 712.954, Valid Time: 0.90 [sec]
2022-09-29 09:55:27.312915 | Epoch: 3 | Train Loss: 637.516, Train Time: 4.27 [sec] | Valid Loss: 609.430, Valid Time: 0.90 [sec]
2022-09-29 09:55:32.678569 | Epoch: 4 | Train Loss: 466.276, Train Time: 4.42 [sec] | Valid Loss: 416.088, Valid Time: 0.90 [sec]
2022-09-29 09:55:37.937783 | Epoch: 5 | Train Loss: 329.638, Train Time: 4.32 [sec] | Valid Loss: 301.526, Valid Time: 0.90 [sec]
2022-09-29 09:55:43.205038 | Epoch: 6 | Train Loss: 244.340, Train Time: 4.33 [sec] | Valid Loss: 241.454, Valid Time: 0.90 [sec]
2022-09-29 09:55:48.505302 | Epoch: 7 | Train Loss: 184.896, Train Time: 4.35 [sec] | Valid Loss: 194.676, Valid Time: 0.91 [sec]
2022-09-29 09:55:53.931883 | Epoch: 8 | Train Loss: 152.953, Train Time: 4.49 [sec] | Vali

In [75]:
n_epochs = 100
random_seed = 1000

dataset_train = STDataset(X_tr, y_tr, time_step=time_step, space_window=space_window)
dataset_valid = STDataset(X_va, y_va, time_step=time_step, space_window=space_window)

model_name = f'zero_fill/TE-ED-S_RS{random_seed}'
log_path = f'./logs/{model_name}.log'

fix_seed(random_seed)

model = T_EncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, dataset_train, dataset_valid, model, 
                model_name, log_path, random_state=random_seed)

print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-29 11:22:34.164888 | Epoch: 1 | Train Loss: 884.054, Train Time: 4.57 [sec] | Valid Loss: 864.634, Valid Time: 0.91 [sec]
2022-09-29 11:22:39.857566 | Epoch: 2 | Train Loss: 733.647, Train Time: 4.75 [sec] | Valid Loss: 728.150, Valid Time: 0.91 [sec]
2022-09-29 11:22:45.426348 | Epoch: 3 | Train Loss: 664.270, Train Time: 4.62 [sec] | Valid Loss: 682.457, Valid Time: 0.91 [sec]
2022-09-29 11:22:50.994573 | Epoch: 4 | Train Loss: 546.073, Train Time: 4.61 [sec] | Valid Loss: 490.116, Valid Time: 0.92 [sec]
2022-09-29 11:22:56.588350 | Epoch: 5 | Train Loss: 383.262, Train Time: 4.64 [sec] | Valid Loss: 399.892, Valid Time: 0.92 [sec]
2022-09-29 11:23:02.181329 | Epoch: 6 | Train Loss: 311.399, Train Time: 4.64 [sec] | Valid Loss: 302.940, Valid Time: 0.92 [sec]
2022-09-29 11:23:07.778343 | Epoch: 7 | Train Loss: 244.765, Train Time: 4.64 [sec] | Valid Loss: 244.791, Valid Time: 0.92 [sec]
2022-09-29 11:23:13.500575 | Epoch: 8 | Train Loss: 199.404, Train Time: 4.77 [sec] | Vali

### 検索あり

In [ ]:
!python3.9 train.py STE-ED-S -d cuda:3 -e 100 -r 1000 --model_name dropout0/text_STE-ED-S_RS1000

============================== Training STE-ED-S for 100 epochs on cuda:3 (seed: 1000) ==============================
2022-10-10 08:18:05.028467 | Epoch: 1 | Train Loss: 886.051, Train Time: 6.40 [sec] | Valid Loss: 863.103, Valid Time: 0.90 [sec]
2022-10-10 08:18:10.240907 | Epoch: 2 | Train Loss: 725.203, Train Time: 4.29 [sec] | Valid Loss: 712.954, Valid Time: 0.90 [sec]
2022-10-10 08:18:15.482537 | Epoch: 3 | Train Loss: 637.516, Train Time: 4.31 [sec] | Valid Loss: 609.430, Valid Time: 0.90 [sec]
2022-10-10 08:18:20.766597 | Epoch: 4 | Train Loss: 466.276, Train Time: 4.32 [sec] | Valid Loss: 416.088, Valid Time: 0.93 [sec]
2022-10-10 08:18:26.065697 | Epoch: 5 | Train Loss: 329.638, Train Time: 4.33 [sec] | Valid Loss: 301.526, Valid Time: 0.93 [sec]
2022-10-10 08:18:31.357442 | Epoch: 6 | Train Loss: 244.340, Train Time: 4.35 [sec] | Valid Loss: 241.454, Valid Time: 0.90 [sec]
2022-10-10 08:18:36.681285 | Epoch: 7 | Train Loss: 184.896, Train Time: 4.38 [sec] | Valid Loss: 194.

### 検索無し

In [ ]:
!python3.9 train.py T-ED-S -d cuda:3 -e 100 -r 1000 --model_name dropout0/text_T-ED_RS1000

In [46]:
n_epochs = 100

model_name = f'dropout0/T-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = LSTMOnlyEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 08:36:43.111458 | Epoch: 1 | Train Loss: 914.648, Train Time: 3.46 [sec] | Valid Loss: 935.555, Valid Time: 0.66 [sec]
2022-09-25 08:36:47.395831 | Epoch: 2 | Train Loss: 813.627, Train Time: 3.61 [sec] | Valid Loss: 803.783, Valid Time: 0.66 [sec]
2022-09-25 08:36:51.554986 | Epoch: 3 | Train Loss: 709.750, Train Time: 3.48 [sec] | Valid Loss: 727.066, Valid Time: 0.66 [sec]
2022-09-25 08:36:55.734122 | Epoch: 4 | Train Loss: 665.993, Train Time: 3.49 [sec] | Valid Loss: 671.653, Valid Time: 0.66 [sec]
2022-09-25 08:36:59.932528 | Epoch: 5 | Train Loss: 575.780, Train Time: 3.51 [sec] | Valid Loss: 539.896, Valid Time: 0.66 [sec]
2022-09-25 08:37:04.151201 | Epoch: 6 | Train Loss: 442.251, Train Time: 3.53 [sec] | Valid Loss: 423.032, Valid Time: 0.66 [sec]
2022-09-25 08:37:08.378018 | Epoch: 7 | Train Loss: 346.851, Train Time: 3.54 [sec] | Valid Loss: 331.522, Valid Time: 0.66 [sec]
2022-09-25 08:37:12.756303 | Epoch: 8 | Train Loss: 266.694, Train Time: 3.69 [sec] | Vali

In [47]:
n_epochs = 100

model_name = f'dropout0/ST-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = CNNLSTMEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 08:44:00.510737 | Epoch: 1 | Train Loss: 911.838, Train Time: 3.72 [sec] | Valid Loss: 929.950, Valid Time: 0.67 [sec]
2022-09-25 08:44:05.075293 | Epoch: 2 | Train Loss: 807.732, Train Time: 3.87 [sec] | Valid Loss: 798.625, Valid Time: 0.68 [sec]
2022-09-25 08:44:09.518703 | Epoch: 3 | Train Loss: 707.269, Train Time: 3.73 [sec] | Valid Loss: 725.903, Valid Time: 0.68 [sec]
2022-09-25 08:44:13.962886 | Epoch: 4 | Train Loss: 654.070, Train Time: 3.74 [sec] | Valid Loss: 640.062, Valid Time: 0.68 [sec]
2022-09-25 08:44:18.414196 | Epoch: 5 | Train Loss: 535.116, Train Time: 3.75 [sec] | Valid Loss: 516.778, Valid Time: 0.68 [sec]
2022-09-25 08:44:22.998682 | Epoch: 6 | Train Loss: 412.359, Train Time: 3.89 [sec] | Valid Loss: 393.887, Valid Time: 0.68 [sec]
2022-09-25 08:44:27.439095 | Epoch: 7 | Train Loss: 303.933, Train Time: 3.74 [sec] | Valid Loss: 294.893, Valid Time: 0.67 [sec]
2022-09-25 08:44:31.898811 | Epoch: 8 | Train Loss: 245.083, Train Time: 3.75 [sec] | Vali

In [50]:
n_epochs = 100

model_name = f'dropout0/TE-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = LSTMEmbeddingEncoderDecoder(lstm_dropout=0)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 09:36:58.879885 | Epoch: 1 | Train Loss: 914.875, Train Time: 3.76 [sec] | Valid Loss: 938.704, Valid Time: 0.79 [sec]
2022-09-25 09:37:03.322610 | Epoch: 2 | Train Loss: 822.279, Train Time: 3.62 [sec] | Valid Loss: 816.952, Valid Time: 0.79 [sec]
2022-09-25 09:37:07.781082 | Epoch: 3 | Train Loss: 717.647, Train Time: 3.64 [sec] | Valid Loss: 730.051, Valid Time: 0.79 [sec]
2022-09-25 09:37:12.250469 | Epoch: 4 | Train Loss: 667.605, Train Time: 3.65 [sec] | Valid Loss: 680.037, Valid Time: 0.79 [sec]
2022-09-25 09:37:16.743929 | Epoch: 5 | Train Loss: 583.678, Train Time: 3.67 [sec] | Valid Loss: 554.936, Valid Time: 0.79 [sec]
2022-09-25 09:37:21.370787 | Epoch: 6 | Train Loss: 448.154, Train Time: 3.67 [sec] | Valid Loss: 419.955, Valid Time: 0.93 [sec]
2022-09-25 09:37:25.871545 | Epoch: 7 | Train Loss: 342.910, Train Time: 3.68 [sec] | Valid Loss: 329.363, Valid Time: 0.79 [sec]
2022-09-25 09:37:30.388383 | Epoch: 8 | Train Loss: 267.807, Train Time: 3.70 [sec] | Vali

In [49]:
n_epochs = 100

model_name = f'dropout0/STE-ED_LSTM4_RS{random_seed}'
log_path = f'./logs/{model_name}.log'
    
model = EncoderDecoder(lstm_dropout=0, include_search=False)
trainer = train(n_epochs, model, model_name, log_path, random_state=random_seed)
    
print('='*40, f'best loss: {min(trainer.val_losses):.3f}', '='*40)

2022-09-25 08:59:12.298124 | Epoch: 1 | Train Loss: 902.922, Train Time: 3.89 [sec] | Valid Loss: 908.401, Valid Time: 0.81 [sec]
2022-09-25 08:59:17.009978 | Epoch: 2 | Train Loss: 778.974, Train Time: 3.88 [sec] | Valid Loss: 766.136, Valid Time: 0.81 [sec]
2022-09-25 08:59:21.853420 | Epoch: 3 | Train Loss: 687.037, Train Time: 4.00 [sec] | Valid Loss: 709.010, Valid Time: 0.81 [sec]
2022-09-25 08:59:26.592350 | Epoch: 4 | Train Loss: 604.120, Train Time: 3.90 [sec] | Valid Loss: 567.112, Valid Time: 0.81 [sec]
2022-09-25 08:59:31.331137 | Epoch: 5 | Train Loss: 448.884, Train Time: 3.90 [sec] | Valid Loss: 422.681, Valid Time: 0.81 [sec]
2022-09-25 08:59:36.067184 | Epoch: 6 | Train Loss: 340.401, Train Time: 3.89 [sec] | Valid Loss: 342.251, Valid Time: 0.82 [sec]
2022-09-25 08:59:40.932599 | Epoch: 7 | Train Loss: 280.144, Train Time: 4.03 [sec] | Valid Loss: 268.297, Valid Time: 0.81 [sec]
2022-09-25 08:59:45.667672 | Epoch: 8 | Train Loss: 228.231, Train Time: 3.90 [sec] | Vali

#### dropout

In [29]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/STE-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = EncoderDecoder(lstm_dropout=lstm_dropout, include_search=False)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 12:06:19.729555 | Epoch: 1 | Train Loss: 912.308, Train Time: 4.33 [sec] | Valid Loss: 930.983, Valid Time: 0.95 [sec]
2022-09-21 12:06:24.905756 | Epoch: 2 | Train Loss: 808.207, Train Time: 4.34 [sec] | Valid Loss: 797.684, Valid Time: 0.81 [sec]
2022-09-21 12:06:30.069718 | Epoch: 3 | Train Loss: 704.179, Train Time: 4.33 [sec] | Valid Loss: 719.649, Valid Time: 0.81 [sec]
2022-09-21 12:06:35.240472 | Epoch: 4 | Train Loss: 642.821, Train Time: 4.33 [sec] | Valid Loss: 610.803, Valid Time: 0.81 [sec]
2022-09-21 12:06:40.541625 | Epoch: 5 | Train Loss: 492.357, Train Time: 4.46 [sec] | Valid Loss: 450.967, Valid Time: 0.81 [sec]
2022-09-21 12:06:45.728189 | Epoch: 6 | Train Loss: 344.074, Train Time: 4.34 [sec] | Valid Loss: 324.891, Valid Time: 0.82 [sec]
2022-09-21 12:06:50.909874 | Epoch: 7 | Train Loss: 244.826, Train Time: 4.34 [sec] | Valid Loss: 264.366, Valid Time: 0.81 [sec]
20

In [30]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/T-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = LSTMOnlyEncoderDecoder(lstm_dropout=lstm_dropout)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 13:32:38.742919 | Epoch: 1 | Train Loss: 916.855, Train Time: 4.11 [sec] | Valid Loss: 944.451, Valid Time: 0.68 [sec]
2022-09-21 13:32:43.549089 | Epoch: 2 | Train Loss: 833.289, Train Time: 4.12 [sec] | Valid Loss: 833.622, Valid Time: 0.67 [sec]
2022-09-21 13:32:48.504610 | Epoch: 3 | Train Loss: 733.092, Train Time: 4.25 [sec] | Valid Loss: 745.880, Valid Time: 0.67 [sec]
2022-09-21 13:32:53.328205 | Epoch: 4 | Train Loss: 684.810, Train Time: 4.12 [sec] | Valid Loss: 714.688, Valid Time: 0.67 [sec]
2022-09-21 13:32:58.142882 | Epoch: 5 | Train Loss: 618.394, Train Time: 4.12 [sec] | Valid Loss: 645.846, Valid Time: 0.67 [sec]
2022-09-21 13:33:02.964236 | Epoch: 6 | Train Loss: 510.655, Train Time: 4.12 [sec] | Valid Loss: 504.062, Valid Time: 0.68 [sec]
2022-09-21 13:33:07.770264 | Epoch: 7 | Train Loss: 414.541, Train Time: 4.10 [sec] | Valid Loss: 397.476, Valid Time: 0.68 [sec]
20

In [31]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/ST-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = CNNLSTMEncoderDecoder(lstm_dropout=lstm_dropout)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 14:53:11.428081 | Epoch: 1 | Train Loss: 910.201, Train Time: 4.17 [sec] | Valid Loss: 925.179, Valid Time: 0.68 [sec]
2022-09-21 14:53:16.420540 | Epoch: 2 | Train Loss: 801.009, Train Time: 4.30 [sec] | Valid Loss: 791.218, Valid Time: 0.67 [sec]
2022-09-21 14:53:21.258439 | Epoch: 3 | Train Loss: 702.928, Train Time: 4.13 [sec] | Valid Loss: 723.204, Valid Time: 0.68 [sec]
2022-09-21 14:53:26.187419 | Epoch: 4 | Train Loss: 633.418, Train Time: 4.22 [sec] | Valid Loss: 609.755, Valid Time: 0.68 [sec]
2022-09-21 14:53:31.142880 | Epoch: 5 | Train Loss: 486.849, Train Time: 4.25 [sec] | Valid Loss: 471.773, Valid Time: 0.68 [sec]
2022-09-21 14:53:36.065098 | Epoch: 6 | Train Loss: 348.703, Train Time: 4.21 [sec] | Valid Loss: 353.162, Valid Time: 0.68 [sec]
2022-09-21 14:53:40.916109 | Epoch: 7 | Train Loss: 261.980, Train Time: 4.15 [sec] | Valid Loss: 280.254, Valid Time: 0.67 [sec]
20

In [32]:
n_epochs = 100

n_exp = 10

lstm_layers = 4
lstm_dropout = 0.4

trainer_list = []

for i in range(n_exp):
    model_name = f'dropout{lstm_dropout}/TE-ED_LSTM{lstm_layers}_exp{i+1}'
    log_path = f'./logs/{model_name}.log'
    
    print('='*40, f'Exp.{i+1}', '='*40)
    TRAFFIC_LSTM_LAYERS = SEARCH_LSTM_LAYERS = lstm_layers
    
    model = LSTMEmbeddingEncoderDecoder(lstm_dropout=lstm_dropout)
    trainer = train(n_epochs, model, model_name, log_path)
    
    print('='*40, f'(Exp.{i+1}) best loss: {min(trainer.val_losses):.3f}', '='*40)
    trainer_list.append(trainer)

best_losses = [min(trainer.val_losses) for trainer in trainer_list]
print(f'Mean MAE: {sum(best_losses) / n_exp:.3f}')

======================================== Exp.1 ========================================
2022-09-21 16:14:40.505355 | Epoch: 1 | Train Loss: 912.275, Train Time: 4.25 [sec] | Valid Loss: 927.834, Valid Time: 0.81 [sec]
2022-09-21 16:14:45.542849 | Epoch: 2 | Train Loss: 798.748, Train Time: 4.21 [sec] | Valid Loss: 782.714, Valid Time: 0.81 [sec]
2022-09-21 16:14:50.593549 | Epoch: 3 | Train Loss: 694.358, Train Time: 4.22 [sec] | Valid Loss: 712.884, Valid Time: 0.81 [sec]
2022-09-21 16:14:55.752523 | Epoch: 4 | Train Loss: 643.735, Train Time: 4.33 [sec] | Valid Loss: 661.820, Valid Time: 0.80 [sec]
2022-09-21 16:15:00.819255 | Epoch: 5 | Train Loss: 603.858, Train Time: 4.24 [sec] | Valid Loss: 555.274, Valid Time: 0.80 [sec]
2022-09-21 16:15:05.879349 | Epoch: 6 | Train Loss: 478.532, Train Time: 4.23 [sec] | Valid Loss: 616.711, Valid Time: 0.80 [sec]
2022-09-21 16:15:10.903636 | Epoch: 7 | Train Loss: 319.604, Train Time: 4.22 [sec] | Valid Loss: 694.529, Valid Time: 0.81 [sec]
20